In [2]:
import numpy as np
import pandas as pd
import sklearn as sk

In [3]:
df = pd.read_csv('BankChurners.csv')
df.head()

,CLIENTNUM,Attrition_Flag,Customer_Age,Gender,Dependent_count,Education_Level,Marital_Status,Income_Category,Card_Category,Months_on_book,...,Credit_Limit,Total_Revolving_Bal,Avg_Open_To_Buy,Total_Amt_Chng_Q4_Q1,Total_Trans_Amt,Total_Trans_Ct,Total_Ct_Chng_Q4_Q1,Avg_Utilization_Ratio,Naive_Bayes_Classifier_Attrition_Flag_Card_Category_Contacts_Count_12_mon_Dependent_count_Education_Level_Months_Inactive_12_mon_1,Naive_Bayes_Classifier_Attrition_Flag_Card_Category_Contacts_Count_12_mon_Dependent_count_Education_Level_Months_Inactive_12_mon_2
0,768805383,Existing Customer,45,M,3,High School,Married,$60K - $80K,Blue,39,...,12691.0,777,11914.0,1.335,1144,42,1.625,0.061,0.000093,0.99991
1,818770008,Existing Customer,49,F,5,Graduate,Single,Less than $40K,Blue,44,...,8256.0,864,7392.0,1.541,1291,33,3.714,0.105,0.000057,0.99994
2,713982108,Existing Customer,51,M,3,Graduate,Married,$80K - $120K,Blue,36,...,3418.0,0,3418.0,2.594,1887,20,2.333,0.000,0.000021,0.99998
3,769911858,Existing Customer,40,F,4,High School,Unknown,Less than $40K,Blue,34,...,3313.0,2517,796.0,1.405,1171,20,2.333,0.760,0.000134,0.99987
4,709106358,Existing Customer,40,M,3,Uneducated,Married,$60K - $80K,Blue,21,...,4716.0,0,4716.0,2.175,816,28,2.500,0.000,0.000022,0.99998


In [4]:
# Delete the last two columns
df = df.iloc[:, :-2]

In [5]:
# We find that there exist 'Unknown' in some features, we need to transfer the 'Unknown' to 'Nan'
df = df.replace({'Unknown': np.nan})
df.isnull().sum()

CLIENTNUM                      0
Attrition_Flag                 0
Customer_Age                   0
Gender                         0
Dependent_count                0
Education_Level             1519
Marital_Status               749
Income_Category             1112
Card_Category                  0
Months_on_book                 0
Total_Relationship_Count       0
Months_Inactive_12_mon         0
Contacts_Count_12_mon          0
Credit_Limit                   0
Total_Revolving_Bal            0
Avg_Open_To_Buy                0
Total_Amt_Chng_Q4_Q1           0
Total_Trans_Amt                0
Total_Trans_Ct                 0
Total_Ct_Chng_Q4_Q1            0
Avg_Utilization_Ratio          0
dtype: int64

In [6]:
unique_counts = {}
for column in df.columns:
    unique_counts[column] = df[column].nunique()

# print the number of unique, which can help us identify the discrete variable
for column, count in unique_counts.items():
    print(f"Number of unique values in column '{column}': {count}")


Number of unique values in column 'CLIENTNUM': 10127
Number of unique values in column 'Attrition_Flag': 2
Number of unique values in column 'Customer_Age': 45
Number of unique values in column 'Gender': 2
Number of unique values in column 'Dependent_count': 6
Number of unique values in column 'Education_Level': 6
Number of unique values in column 'Marital_Status': 3
Number of unique values in column 'Income_Category': 5
Number of unique values in column 'Card_Category': 4
Number of unique values in column 'Months_on_book': 44
Number of unique values in column 'Total_Relationship_Count': 6
Number of unique values in column 'Months_Inactive_12_mon': 7
Number of unique values in column 'Contacts_Count_12_mon': 7
Number of unique values in column 'Credit_Limit': 6205
Number of unique values in column 'Total_Revolving_Bal': 1974
Number of unique values in column 'Avg_Open_To_Buy': 6813
Number of unique values in column 'Total_Amt_Chng_Q4_Q1': 1158
Number of unique values in column 'Total_T

In [7]:
discrete_feature = ['Dependent_count', 'Education_Level', 'Income_Category', 'Total_Relationship_Count', 'Months_Inactive_12_mon', 'Contacts_Count_12_mon']

## Data Processing

In [8]:
#we directly drop the nan sample.
df.dropna(inplace=True)
len(df)

7081

In [9]:
from sklearn import preprocessing
label_encoder = preprocessing.LabelEncoder()
df['Attrition_Flag'] = df['Attrition_Flag'].replace({'Existing Customer': 0, 'Attrited Customer':1})
df['Gender'] = df['Gender'].replace({'M': 0, 'F':1})
salary_transfer ={'Less than $40K': 0, '$40K - $60K': 1, '$60K - $80K': 2, '$80K - $120K': 3, '$120K +': 4}
education_transfer = {'Uneducated': 0, 'High School':1, 'College': 2, 'Graduate': 3, 'Post-Graduate': 4, 'Doctorate':5}
df['Credit_Limit'] = np.log(df['Credit_Limit'])
df['Total_Trans_Amt'] = np.log(df['Total_Trans_Amt'])
df['Income_Category'] = df['Income_Category'].replace(salary_transfer)
df['Education_Level'] = df['Education_Level'].replace(education_transfer)
df['Marital_Status'] = label_encoder.fit_transform(df['Marital_Status'])
df.drop('Card_Category', axis=1, inplace=True)
df

,CLIENTNUM,Attrition_Flag,Customer_Age,Gender,Dependent_count,Education_Level,Marital_Status,Income_Category,Months_on_book,Total_Relationship_Count,Months_Inactive_12_mon,Contacts_Count_12_mon,Credit_Limit,Total_Revolving_Bal,Avg_Open_To_Buy,Total_Amt_Chng_Q4_Q1,Total_Trans_Amt,Total_Trans_Ct,Total_Ct_Chng_Q4_Q1,Avg_Utilization_Ratio
0,768805383,0,45,0,3,1,1,2,39,5,1,3,9.448648,777,11914.0,1.335,7.042286,42,1.625,0.061
1,818770008,0,49,1,5,3,2,0,44,6,1,2,9.018695,864,7392.0,1.541,7.163172,33,3.714,0.105
2,713982108,0,51,0,3,3,1,3,36,4,1,0,8.136811,0,3418.0,2.594,7.542744,20,2.333,0.000
4,709106358,0,40,0,3,0,1,2,21,5,1,0,8.458716,0,4716.0,2.175,6.704414,28,2.500,0.000
5,713061558,0,44,0,2,3,1,1,36,3,1,2,8.296547,1247,2763.0,1.376,6.992096,24,0.846,0.311
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10120,710841183,0,54,0,1,1,2,2,34,5,2,0,9.542518,2109,11831.0,0.660,9.653551,114,0.754,0.151
10121,713899383,0,56,1,1,3,2,0,50,4,1,4,8.212840,606,3082.0,0.570,9.588503,120,0.791,0.164
10122,772366833,0,50,0,2,3,2,1,40,3,2,3,8.294799,1851,2152.0,0.703,9.647046,117,0.857,0.462
10124,716506083,1,44,1,1,1,1,0,36,5,3,4,8.595820,0,5409.0,0.819,9.239025,60,0.818,0.000


In [10]:
df1 = df.copy()

## Data Generlization

In [11]:
def feature_generation(df, cats, features, types):
    for cat in cats:
        for feature in features:
            stat = df.groupby(cat)[feature].agg(types)
            col_names = list()
            for type in types:
                col_name = cat+'_'+ feature + '_' + type
                col_names.append(col_name)
            stat.columns = col_names
            df = pd.merge(df, stat, on=cat)

    return df 

In [12]:
cats = ['Income_Category','Education_Level', 'Gender', 'Marital_Status']
c_features = ['Total_Trans_Amt', 'Total_Trans_Ct', 'Total_Revolving_Bal', 'Credit_Limit']
c_types = ['mean', 'max', 'min']
df1 = feature_generation(df1, cats=cats, features=c_features, types = c_types)
df1

,CLIENTNUM,Attrition_Flag,Customer_Age,Gender,Dependent_count,Education_Level,Marital_Status,Income_Category,Months_on_book,Total_Relationship_Count,...,Marital_Status_Total_Trans_Amt_min,Marital_Status_Total_Trans_Ct_mean,Marital_Status_Total_Trans_Ct_max,Marital_Status_Total_Trans_Ct_min,Marital_Status_Total_Revolving_Bal_mean,Marital_Status_Total_Revolving_Bal_max,Marital_Status_Total_Revolving_Bal_min,Marital_Status_Credit_Limit_mean,Marital_Status_Credit_Limit_max,Marital_Status_Credit_Limit_min
0,768805383,0,45,0,3,1,1,2,39,5,...,6.234411,61.358866,131,10,1207.743266,2517,0,8.536945,10.449178,7.271217
1,778493808,0,49,0,3,1,1,2,37,5,...,6.234411,61.358866,131,10,1207.743266,2517,0,8.536945,10.449178,7.271217
2,709531908,0,53,0,3,1,1,2,47,5,...,6.234411,61.358866,131,10,1207.743266,2517,0,8.536945,10.449178,7.271217
3,712701333,0,53,0,3,1,1,2,43,5,...,6.234411,61.358866,131,10,1207.743266,2517,0,8.536945,10.449178,7.271217
4,788194458,0,34,0,3,1,1,2,26,4,...,6.234411,61.358866,131,10,1207.743266,2517,0,8.536945,10.449178,7.271217
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7076,717783483,0,47,1,5,4,0,0,30,4,...,6.660575,67.286467,127,20,1172.101933,2517,0,8.638233,10.449178,7.271217
7077,721276383,0,47,1,2,4,0,1,33,6,...,6.660575,67.286467,127,20,1172.101933,2517,0,8.638233,10.449178,7.271217
7078,711428733,0,50,1,3,4,0,1,39,3,...,6.660575,67.286467,127,20,1172.101933,2517,0,8.638233,10.449178,7.271217
7079,803197833,0,49,1,1,4,0,1,44,6,...,6.660575,67.286467,127,20,1172.101933,2517,0,8.638233,10.449178,7.271217


In [13]:
X = df.drop(['CLIENTNUM','Attrition_Flag'],axis=1)
y = df.Attrition_Flag

In [14]:
# In original data, the Attrition sample just accounts for about 16%. The data is imbalanced.
y.sum()/len(y)

0.15718118909758508

## MLP Classifiers

In [15]:
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score, f1_score, recall_score

def train_and_evaluate(X_train, y_train, X_test, y_test):

    mlp_classifier = MLPClassifier(random_state=42)
    
    mlp_classifier.fit(X_train, y_train)

    y_pred = mlp_classifier.predict(X_test)

    accuracy = accuracy_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred)
    recall = recall_score(y_test, y_pred)
    
    return accuracy, f1, recall

### GridSearch

In [16]:
from sklearn.model_selection import GridSearchCV

def grid_search_mlp(X_train, y_train, X_test, y_test):

    mlp_classifier = MLPClassifier(random_state=42)
    

    param_grid = {
        'hidden_layer_sizes': [(50,), (100,), (50, 50), (100, 50)],
        'activation': ['relu', 'tanh'],
        'solver': ['adam', 'sgd'],
        'alpha': [0.0001, 0.001, 0.01, 0.1],
    }

    grid_search = GridSearchCV(mlp_classifier, param_grid, scoring='f1', cv=3)

    grid_search.fit(X_train, y_train)
    

    best_mlp_model = grid_search.best_estimator_
    

    y_pred = best_mlp_model.predict(X_test)
    

    accuracy = accuracy_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred)
    recall = recall_score(y_test, y_pred)
    
    return best_mlp_model, accuracy, f1, recall

### NoGeneralization NoNormalization

In [17]:
from sklearn.model_selection import train_test_split
X_origin_train, X_test, y_origin_train, y_test= train_test_split(X,y, random_state=42,test_size=0.2, stratify=y, shuffle=True)
from imblearn.combine import SMOTEENN
smote_enn = SMOTEENN(random_state=42)
X_resampled, y_resampled = smote_enn.fit_resample(X_origin_train, y_origin_train)

In [17]:
accuracy_original, f1_original, recall_original = train_and_evaluate(X_origin_train, y_origin_train, X_test, y_test)

accuracy_resampled, f1_resampled, recall_resampled = train_and_evaluate(X_resampled, y_resampled, X_test, y_test)

print("----NoGen NoNorm original results----")
print("Accuracy:", accuracy_original)
print("F1-Score:", f1_original)
print("Recall:", recall_original)
print("\n")

print("----NoGen NoNorm resampled results----")
print("Accuracy:", accuracy_resampled)
print("F1-Score:", f1_resampled)
print("Recall:", recall_resampled)

----NoGen NoNorm original results----
Accuracy: 0.8659139026111503
F1-Score: 0.3402777777777778
Recall: 0.21973094170403587


----NoGen NoNorm resampled results----
Accuracy: 0.8383909668313339
F1-Score: 0.5336048879837069
Recall: 0.5874439461883408


In [18]:
best_mlp_model_original, accuracy_original_grid, f1_original_grid, recall_original_grid = grid_search_mlp(X_origin_train, y_origin_train, X_test, y_test)

d:\python\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
d:\python\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
d:\python\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
d:\python\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


In [19]:
print("MLP NoGen NoNorm original：")
print("BEST p:", best_mlp_model_original.get_params())
print("Accuracy:", accuracy_original_grid)
print("F1-Score:", f1_original_grid)
print("Recall:", recall_original_grid)

MLP NoGen NoNorm original：
BEST p: {'activation': 'relu', 'alpha': 0.0001, 'batch_size': 'auto', 'beta_1': 0.9, 'beta_2': 0.999, 'early_stopping': False, 'epsilon': 1e-08, 'hidden_layer_sizes': (50,), 'learning_rate': 'constant', 'learning_rate_init': 0.001, 'max_fun': 15000, 'max_iter': 200, 'momentum': 0.9, 'n_iter_no_change': 10, 'nesterovs_momentum': True, 'power_t': 0.5, 'random_state': 42, 'shuffle': True, 'solver': 'adam', 'tol': 0.0001, 'validation_fraction': 0.1, 'verbose': False, 'warm_start': False}
Accuracy: 0.8694424841213833
F1-Score: 0.3934426229508196
Recall: 0.26905829596412556


In [20]:
best_mlp_model_resampled, accuracy_resampled_grid, f1_resampled_grid, recall_resampled_grid = grid_search_mlp(X_resampled, y_resampled, X_test, y_test)

In [21]:
print("MLP NoGen NoNorm resampled：")
print("BEST p:", best_mlp_model_resampled.get_params())
print("Accuracy:", accuracy_resampled_grid)
print("F1-Score:", f1_resampled_grid)
print("Recall:", recall_resampled_grid)

MLP NoGen NoNorm resampled：
BEST p: {'activation': 'relu', 'alpha': 0.0001, 'batch_size': 'auto', 'beta_1': 0.9, 'beta_2': 0.999, 'early_stopping': False, 'epsilon': 1e-08, 'hidden_layer_sizes': (50,), 'learning_rate': 'constant', 'learning_rate_init': 0.001, 'max_fun': 15000, 'max_iter': 200, 'momentum': 0.9, 'n_iter_no_change': 10, 'nesterovs_momentum': True, 'power_t': 0.5, 'random_state': 42, 'shuffle': True, 'solver': 'adam', 'tol': 0.0001, 'validation_fraction': 0.1, 'verbose': False, 'warm_start': False}
Accuracy: 0.7854622441778405
F1-Score: 0.5220125786163522
Recall: 0.7443946188340808


### NoGeneralization Normalization

In [18]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X_origin_train_scaled = scaler.fit_transform(X_origin_train)
X_resampled_scaled = scaler.fit_transform(X_resampled)
X_test_scaled = scaler.transform(X_test)

accuracy_original_scaled, f1_original_scaled, recall_original_scaled = train_and_evaluate(X_origin_train_scaled, y_origin_train, X_test_scaled, y_test)

accuracy_resampled_scaled, f1_resampled_scaled, recall_resampled_scaled = train_and_evaluate(X_resampled_scaled, y_resampled, X_test_scaled, y_test)

print("----NoGen Norm original results----")
print("Accuracy:", accuracy_original_scaled)
print("F1-Score:", f1_original_scaled)
print("Recall:", recall_original_scaled)
print("\n")

print("----NoGen Norm resampled results----")
print("Accuracy:", accuracy_resampled_scaled)
print("F1-Score:", f1_resampled_scaled)
print("Recall:", recall_resampled_scaled)


d:\python\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


----NoGen Norm original results----
Accuracy: 0.9322512350035286
F1-Score: 0.734806629834254
Recall: 0.5964125560538116


----NoGen Norm resampled results----
Accuracy: 0.8920254057868737
F1-Score: 0.6982248520710058
Recall: 0.7937219730941704


d:\python\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


In [19]:
best_mlp_model_original, accuracy_original_grid, f1_original_grid, recall_original_grid = grid_search_mlp(X_origin_train_scaled, y_origin_train, X_test_scaled, y_test)

d:\python\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
d:\python\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
d:\python\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
d:\python\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
d:\python\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum 

In [20]:
print("MLP NoGen Norm original：")
print("BEST p:", best_mlp_model_original.get_params())
print("Accuracy:", accuracy_original_grid)
print("F1-Score:", f1_original_grid)
print("Recall:", recall_original_grid)

MLP NoGen Norm original：
BEST p: {'activation': 'tanh', 'alpha': 0.1, 'batch_size': 'auto', 'beta_1': 0.9, 'beta_2': 0.999, 'early_stopping': False, 'epsilon': 1e-08, 'hidden_layer_sizes': (100, 50), 'learning_rate': 'constant', 'learning_rate_init': 0.001, 'max_fun': 15000, 'max_iter': 200, 'momentum': 0.9, 'n_iter_no_change': 10, 'nesterovs_momentum': True, 'power_t': 0.5, 'random_state': 42, 'shuffle': True, 'solver': 'adam', 'tol': 0.0001, 'validation_fraction': 0.1, 'verbose': False, 'warm_start': False}
Accuracy: 0.9520112914608327
F1-Score: 0.826530612244898
Recall: 0.726457399103139


In [21]:
best_mlp_model_resampled, accuracy_resampled_grid, f1_resampled_grid, recall_resampled_grid = grid_search_mlp(X_resampled_scaled, y_resampled, X_test_scaled, y_test)

d:\python\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
d:\python\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
d:\python\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
d:\python\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
d:\python\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum 

In [22]:
print("MLP NoGen Norm resampled：")
print("BEST p:", best_mlp_model_resampled.get_params())
print("Accuracy:", accuracy_resampled_grid)
print("F1-Score:", f1_resampled_grid)
print("Recall:", recall_resampled_grid)

MLP NoGen Norm resampled：
BEST p: {'activation': 'tanh', 'alpha': 0.0001, 'batch_size': 'auto', 'beta_1': 0.9, 'beta_2': 0.999, 'early_stopping': False, 'epsilon': 1e-08, 'hidden_layer_sizes': (100,), 'learning_rate': 'constant', 'learning_rate_init': 0.001, 'max_fun': 15000, 'max_iter': 200, 'momentum': 0.9, 'n_iter_no_change': 10, 'nesterovs_momentum': True, 'power_t': 0.5, 'random_state': 42, 'shuffle': True, 'solver': 'adam', 'tol': 0.0001, 'validation_fraction': 0.1, 'verbose': False, 'warm_start': False}
Accuracy: 0.9011997177134792
F1-Score: 0.7154471544715446
Recall: 0.7892376681614349


### Generalization NoNormalization

In [23]:
X = df1.drop(['CLIENTNUM','Attrition_Flag'],axis=1)
y = df1.Attrition_Flag
X_origin_train, X_test, y_origin_train, y_test= train_test_split(X,y, random_state=42,test_size=0.2, stratify=y, shuffle=True)
from imblearn.combine import SMOTEENN
smote_enn = SMOTEENN(random_state=42)
X_resampled, y_resampled = smote_enn.fit_resample(X_origin_train, y_origin_train)

In [24]:
accuracy_original, f1_original, recall_original = train_and_evaluate(X_origin_train, y_origin_train, X_test, y_test)
accuracy_resampled, f1_resampled, recall_resampled = train_and_evaluate(X_resampled, y_resampled, X_test, y_test)

print("----Gen NoNorm original results----")
print("Accuracy:", accuracy_original)
print("F1-Score:", f1_original)
print("Recall:", recall_original)
print("\n")

print("----Gen NoNorm resampled results----")
print("Accuracy:", accuracy_resampled)
print("F1-Score:", f1_resampled)
print("Recall:", recall_resampled)


----Gen NoNorm original results----
Accuracy: 0.8666196189131969
F1-Score: 0.35051546391752575
Recall: 0.22869955156950672


----Gen NoNorm resampled results----
Accuracy: 0.5010585744530699
F1-Score: 0.37709251101321584
Recall: 0.9596412556053812


In [25]:
best_mlp_model_original, accuracy_original_grid, f1_original_grid, recall_original_grid = grid_search_mlp(X_origin_train, y_origin_train, X_test, y_test)

In [26]:
print("MLP Gen NoNorm original：")
print("BEST p:", best_mlp_model_original.get_params())
print("Accuracy:", accuracy_original_grid)
print("F1-Score:", f1_original_grid)
print("Recall:", recall_original_grid)

MLP Gen NoNorm original：
BEST p: {'activation': 'relu', 'alpha': 0.01, 'batch_size': 'auto', 'beta_1': 0.9, 'beta_2': 0.999, 'early_stopping': False, 'epsilon': 1e-08, 'hidden_layer_sizes': (50,), 'learning_rate': 'constant', 'learning_rate_init': 0.001, 'max_fun': 15000, 'max_iter': 200, 'momentum': 0.9, 'n_iter_no_change': 10, 'nesterovs_momentum': True, 'power_t': 0.5, 'random_state': 42, 'shuffle': True, 'solver': 'adam', 'tol': 0.0001, 'validation_fraction': 0.1, 'verbose': False, 'warm_start': False}
Accuracy: 0.8779110797459422
F1-Score: 0.5575447570332481
Recall: 0.48878923766816146


In [27]:
best_mlp_model_resampled, accuracy_resampled_grid, f1_resampled_grid, recall_resampled_grid = grid_search_mlp(X_resampled, y_resampled, X_test, y_test)

In [28]:
print("MLP NoGen NoNorm resampled：")
print("BEST p:", best_mlp_model_resampled.get_params())
print("Accuracy:", accuracy_resampled_grid)
print("F1-Score:", f1_resampled_grid)
print("Recall:", recall_resampled_grid)

MLP NoGen NoNorm resampled：
BEST p: {'activation': 'relu', 'alpha': 0.1, 'batch_size': 'auto', 'beta_1': 0.9, 'beta_2': 0.999, 'early_stopping': False, 'epsilon': 1e-08, 'hidden_layer_sizes': (50,), 'learning_rate': 'constant', 'learning_rate_init': 0.001, 'max_fun': 15000, 'max_iter': 200, 'momentum': 0.9, 'n_iter_no_change': 10, 'nesterovs_momentum': True, 'power_t': 0.5, 'random_state': 42, 'shuffle': True, 'solver': 'adam', 'tol': 0.0001, 'validation_fraction': 0.1, 'verbose': False, 'warm_start': False}
Accuracy: 0.7198306280875089
F1-Score: 0.49683143219264886
Recall: 0.8789237668161435


### Generalization Normalization

In [29]:
scaler = StandardScaler()
X_origin_train_scaled = scaler.fit_transform(X_origin_train)
X_resampled_scaled = scaler.fit_transform(X_resampled)
X_test_scaled = scaler.transform(X_test)

accuracy_original_scaled, f1_original_scaled, recall_original_scaled = train_and_evaluate(X_origin_train_scaled, y_origin_train, X_test_scaled, y_test)

accuracy_resampled_scaled, f1_resampled_scaled, recall_resampled_scaled = train_and_evaluate(X_resampled_scaled, y_resampled, X_test_scaled, y_test)

print("----Gen Norm original results----")
print("Accuracy:", accuracy_original_scaled)
print("F1-Score:", f1_original_scaled)
print("Recall:", recall_original_scaled)
print("\n")

print("----Gen Norm resampled results----")
print("Accuracy:", accuracy_resampled_scaled)
print("F1-Score:", f1_resampled_scaled)
print("Recall:", recall_resampled_scaled)


d:\python\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


----Gen Norm original results----
Accuracy: 0.8962597035991532
F1-Score: 0.5303514376996805
Recall: 0.3721973094170404


----Gen Norm resampled results----
Accuracy: 0.903316866619619
F1-Score: 0.7127882599580712
Recall: 0.7623318385650224


d:\python\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


In [30]:
best_mlp_model_original, accuracy_original_grid, f1_original_grid, recall_original_grid = grid_search_mlp(X_origin_train_scaled, y_origin_train, X_test_scaled, y_test)

d:\python\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
d:\python\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
d:\python\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
d:\python\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
d:\python\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum 

In [31]:
print("MLP Gen Norm original：")
print("BEST p:", best_mlp_model_original.get_params())
print("Accuracy:", accuracy_original_grid)
print("F1-Score:", f1_original_grid)
print("Recall:", recall_original_grid)

MLP Gen Norm original：
BEST p: {'activation': 'relu', 'alpha': 0.1, 'batch_size': 'auto', 'beta_1': 0.9, 'beta_2': 0.999, 'early_stopping': False, 'epsilon': 1e-08, 'hidden_layer_sizes': (100,), 'learning_rate': 'constant', 'learning_rate_init': 0.001, 'max_fun': 15000, 'max_iter': 200, 'momentum': 0.9, 'n_iter_no_change': 10, 'nesterovs_momentum': True, 'power_t': 0.5, 'random_state': 42, 'shuffle': True, 'solver': 'adam', 'tol': 0.0001, 'validation_fraction': 0.1, 'verbose': False, 'warm_start': False}
Accuracy: 0.908256880733945
F1-Score: 0.5987654320987654
Recall: 0.4349775784753363


In [32]:
best_mlp_model_resampled, accuracy_resampled_grid, f1_resampled_grid, recall_resampled_grid = grid_search_mlp(X_resampled_scaled, y_resampled, X_test_scaled, y_test)

d:\python\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
d:\python\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
d:\python\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
d:\python\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
d:\python\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum 

In [33]:
print("MLP Gen Norm resampled：")
print("BEST p:", best_mlp_model_resampled.get_params())
print("Accuracy:", accuracy_resampled_grid)
print("F1-Score:", f1_resampled_grid)
print("Recall:", recall_resampled_grid)

MLP Gen Norm resampled：
BEST p: {'activation': 'tanh', 'alpha': 0.1, 'batch_size': 'auto', 'beta_1': 0.9, 'beta_2': 0.999, 'early_stopping': False, 'epsilon': 1e-08, 'hidden_layer_sizes': (100,), 'learning_rate': 'constant', 'learning_rate_init': 0.001, 'max_fun': 15000, 'max_iter': 200, 'momentum': 0.9, 'n_iter_no_change': 10, 'nesterovs_momentum': True, 'power_t': 0.5, 'random_state': 42, 'shuffle': True, 'solver': 'adam', 'tol': 0.0001, 'validation_fraction': 0.1, 'verbose': False, 'warm_start': False}
Accuracy: 0.9167254763585039
F1-Score: 0.75
Recall: 0.7937219730941704
